In [ ]:
First an approach with 

In [1]:
import math
from collections import namedtuple
import numpy as np
from itertools import pairwise
from aocutils.common import ints
import z3
from z3 import *

Point = namedtuple('Point', 'x y z')
roundpoint = lambda p: Point(round(p.x), round(p.y), round(p.z))

class Line():
    def __init__(self,x=0,y=0,z=0, dx=0,dy=0,dz=0):
        self.x = x
        self.y = y
        self.z = z
        self.dx = dx
        self.dy = dy
        self.dz = dz
        self.update()
        
        
    def update(self):
        self.loc = np.array([self.x, self.y, self.z])
        self.vector = np.array([self.dx, self.dy, self.dz])
        self.point_vector_to_general_form3d()
        
    def point_vector_to_general_form3d(self):
        # Coefficients for the general form ax + by + cz + d = 0
            self.a = self.dy - self.dz
            self.b = self.dz - self.dx
            self.c = self.dx - self.dy
            self.d = self.dx * self.y - self.dy * self.x + self.dz * self.x - self.dx * self.z
            
    def __repr__(self):
        return f'x:{self.x}, y:{self.y}, z:{self.z}, dx:{self.dx}, dy:{self.dy}, dz:{self.dz}'


    def angle(self, other):
        # returns the angle in radians. Divide by 2pi * 360 to get degrees
        # https://www.cuemath.com/geometry/angle-between-vectors/
        dotproduct = self.vector @ other.vector
        a, b = np.linalg.norm(self.vector), np.linalg.norm(other.vector)
        if not a or not b:
            return 0
        return math.acos(np.clip(dotproduct / (a * b),-1,1))

    
    def is_parallel(self, other):
        return np.isclose(self.angle(other) % math.pi, 0, atol=0.0001)

    
    def canreach(self, x1, dx, x2):
        if dx == 0:
            return x1 == x2
        
        
    
    def reachespoint(self,point):
        # returns duration before reaching a point
        # if any of the numbers in the dx are 0
        
        reachable = [x1 == x2 for x1, dx, x2 in zip(self.loc, self.vector, point) if not dx]
        
        if not all(reachable):
            print(reachable)
            return False
        times = [np.round((x2 - x1) / dx,1) for x1, dx, x2 in zip(self.loc, self.vector, point) if dx]
        duration = times[0]
        for y in times[1:]:
            if abs(duration - y) > 0.11:
                print(times)
                return False
        return duration

    
    def samepoint(self, other):
        return self.loc == other.loc
        

    def collide(self, other):
        # collision: t must be the same for x, y and z equations
        # x + t*dx = x2 + t*dx2
        # t*dx - tdx2 = x2 - x
        # t (dx - dx2) = x2 - x
        if self.is_parallel(other):
            print('parallel or same line')
            return False
        coefficients_matrix = [self.dx - other.dx, self.dy - other.dy, self.dz - other.dz]
        constants_vector = [other.x - self.x, other.y - self.y, other.z - self.z]
        ans = {cons / coef for coef, cons in zip(coefficients_matrix, constants_vector) if coef}
        if len(ans) > 1:
            print('no collision')
            return False
        else:
            return ans.pop()
    
    def is_still(self):
        return self.dx == 0 and self.dy == 0



    def intersect(self, other):
        # return t0 and t1 for the two times the lines are at the point and the point where it will intersect as a namedtuple
        # the problem is that leastsq method is quite inaccurate. Therefore use linalg.solve. That however needs square coeffient matrix. Calculate for x and y and check z later with isclose
        if self.is_parallel(other):
            print('parallel or same line')
            return False
        
        
        if self.is_still():
            if self.canreach(other.x, other.dx, self.x):
                t0 = (self.x - other.x) / other.dx
                t1 = t0
            else:
                return False
        elif other.is_still():
            if other.canreach(self.x, self.dx, other.x):
                t0 = (other.x - self.x) / self.dx
                t1 = t0
            else:
                return False
        else:
            coefficients_matrix = np.array([[self.dx, -other.dx], [self.dy, -other.dy]])
            constants_vector = np.array([other.x - self.x, other.y - self.y])

            # intersection: t doesn't need to be equal for line 1 and line 2, but coordinates must match
            # x + t1*dx = x2 + t2*dx2
            # y + t1*dy = y2 + t2*dy2
            # print(coefficients_matrix, constants_vector, self, other)
            t0, t1 = np.linalg.solve(coefficients_matrix, constants_vector)
            t0 = round(t0)
            t1 = round(t1)
        p1 =  np.array([self.x + self.dx * t0, self.y + self.dy *t0, self.z + self.dz * t0])
        p2 =  np.array([other.x + other.dx * t1, other.y + other.dy *t1, other.z + other.dz * t1])     
        return (t0,t1, Point(*p1)) if np.isclose(p1,p2).all() else False

In [2]:
# this approach is based upon finding for each ax hailstones with same velocities. You then know something about the speed of your rock. The distance between the hailstones need to be divisable by the speed of the hailstones (otherwise your rock will never hit the second hailstones)

lines = [Line(*ints(line)) for line in open('in.txt').read().split('\n')]
mods = []
for i in range(3):
    distancespeed = []
    for idx, l1 in enumerate(lines):
        for l2 in lines[idx+1:]:
            if l1.vector[i] == l2.vector[i]:
                distancespeed.append((l1.loc[i] - l2.loc[i], l1.vector[i]))
    mods.append(distancespeed)
vector = []
for mod in mods:
    # DistanceDifference % (RockVelocity-HailVelocity)
    # https://www.reddit.com/r/adventofcode/comments/18pnycy/2023_day_24_solutions
    options = [{rx for rx in range(-1000,1000) if not x % (rx-hx)} for x,hx in mod]
    vector.append(set.intersection(*options).pop())
l = Line(0,0,0,*vector)
for l2 in lines:
    l2.dx -= vector[0]
    l2.dy -= vector[1]
    l2.dz -= vector[2]
    l2.update()
def total_intersection():
    ans = lines[0].intersect(lines[1])
    if not ans:
        return False
    _,_,ans = ans
    ans = roundpoint(ans)
    for l1, l2 in pairwise(lines):
        res = l1.intersect(l2)
        if res is not False:
            _,_,point = res
            if ans != roundpoint(point):
                return False
        else:
            return False
    return ans
sum(total_intersection()) 

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_8528\3552796987.py:16: RuntimeWarning: divide by zero encountered in scalar remainder
  options = [{rx for rx in range(-1000,1000) if not x % (rx-hx)} for x,hx in mod]


673641951253289

In [3]:

lines = [Line(*ints(line)) for line in open('in.txt').read().split('\n')]
ans = 0
minn, maxx = 7, 27
minn, maxx = 200000000000000, 400000000000000
for line in lines:
    line.z = 0
    line.dz = 0
    line.update()

for idx, l1 in enumerate(lines):
    for l2 in lines[idx+1:]:
        res = l1.intersect(l2)
        if res is not False:
            t0,t1,point = res
            if minn <= point.x <= maxx and minn <= point.y <= maxx:
                if t0 >=0 and t1 >= 0:
                    ans += 1
        else:
            # print(l1,l2)
            pass
ans


parallel or same line
parallel or same line
parallel or same line


16502

In [5]:
# this is based upon the fact that there are two hailstones with same z velocity and z position. That means the rock also needs the same z position and velocity.
# Then we do a bruteforce for x and y, with subtracting the speeds of the rock. This means all the lines should intersect at the same point

minn, maxx = 7, 27
minn, maxx = 200000000000000, 400000000000000

roundpoint = lambda p: Point(round(p.x), round(p.y), round(p.z))

lines = [Line(*ints(line)) for line in open('in.txt').read().split('\n')]
def total_intersection():
    ans = lines[0].intersect(lines[1])
    if not ans:
        return False
    _,_,ans = ans
    ans = roundpoint(ans)
    for l1, l2 in pairwise(lines):
        res = l1.intersect(l2)
        if res is not False:
            _,_,point = res

            if ans != roundpoint(point):
                return False
        else:
            return False
    return ans


dz = 71
for line in lines:
    line.dz -= 71

for dx in range(-80, -50):
    for line in lines:
        line.dx -= dx
        
    if dx % 10 == 0:
        print(dx)
    for dy in range(-300,300):
        for line in lines:
            line.dy -= dy
            line.update()
        # dx =  -78
        # dy = 269
        if res := total_intersection():
            print(dx,dy,dz, total_intersection())
        for line in lines:
            line.dy += dy
    for line in lines:
        line.dx += dx
            
    

-80
-78 269 71 Point(x=318090941338468, y=124187623124113, z=231363386790708)
-70
-60


In [7]:

# this is the way on solving it with Z3 very fast

import z3
from z3 import *
from itertools import chain

X = Real('x')
Y = Real('y')
Z = Real('z')
DX = Real('dx')
DY = Real('dy')
DZ = Real('dz')

s = Tactic('qfnra-nlsat').solver()
for idx, line in enumerate(lines):
    NX, NY, NZ, NDX, NDY, NDZ = chain.from_iterable([line.loc, line.vector])
    
    exec(f't{str(idx)} = Int("t{str(idx)}")')
    
    t = eval(f't{str(idx)}')
    
    s.add(t >= 0)
    s.add(X + t * DX == NX + t * NDX)
    s.add(Y + t * DY == NY + t * NDY)
    s.add(Z + t * DZ == NZ + t * NDZ)

    
    
s.check()
m = s.model()

for var in [X,Y,Z,DX,DY,DZ]:
    print(var, m.eval(var).as_long())
    
sum(m.eval(var).as_long() for var in (X,Y,Z))

x 318090941338468
y 124187623124113
z 231363386790708
dx -33
dy 417
dz 0


In [6]:
# maybe I'll revisit this later. The idea is to have two planes and where they intersect the line of the rock is

def plane_from_two_lines(line1, line2):
    # Extracting parameters for the first line
    x1, y1, z1, dx1, dy1, dz1 = line1
    # Extracting parameters for the second line
    x2, y2, z2, dx2, dy2, dz2 = line2

    # Calculate the direction vectors of the lines
    v1 = (dx1, dy1, dz1)
    v2 = (dx2, dy2, dz2)

    # Calculate the normal vector of the plane using the cross product
    normal_vector = (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0]
    )

    # Coefficients of the plane equation
    a, b, c = normal_vector
    d = -(a * x1 + b * y1 + c * z1)  # Use any point on the plane (e.g., the first point of the first line)

    return a, b, c, d


In [89]:
def plane_from_two_lines(line1, line2):
    # Extracting parameters for the first line
    x1, y1, z1, dx1, dy1, dz1 = line1
    # Extracting parameters for the second line
    x2, y2, z2, dx2, dy2, dz2 = line2

    # Calculate the direction vectors of the lines
    v1 = (dx1, dy1, dz1)
    v2 = (dx2, dy2, dz2)

    # Calculate the normal vector of the plane using the cross product
    normal_vector = (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0]
    )

    # Coefficients of the plane equation
    a, b, c = normal_vector
    d = -(a * x1 + b * y1 + c * z1)  # Use any point on the plane (e.g., the first point of the first line)

    return a, b, c, d

# Example usage:
line1 = (1, 2, 3, 4, 5, 6)
line2 = (7, 8, 9, 10, 11, 12)

plane_coefficients = plane_from_two_lines(line1, line2)
print("Plane equation coefficients:", plane_coefficients)


Plane equation coefficients: (-6, 12, -6, 0)


In [59]:
def line_of_intersection(plane1, plane2):
    # Extract coefficients for the first plane
    a1, b1, c1, d1 = plane1
    # Extract coefficients for the second plane
    a2, b2, c2, d2 = plane2

    # Calculate the direction vector of the line of intersection
    direction_vector = (
        b1 * c2 - c1 * b2,
        c1 * a2 - a1 * c2,
        a1 * b2 - b1 * a2
    )

    # Choose a point on the line (you can adjust this as needed)
    intersection_point = (303867887720968, 410071000835863, 211451111726208)

    # Return the line passing through the intersection point with the calculated direction vector
    return (*intersection_point, *direction_vector)

# Example usage:
plane1 = (1, 2, 3, 0)
plane2 = (4, 5, 6, 0)

intersection_line = line_of_intersection(plane1, plane2)
print("Line of intersection between planes:", intersection_line)


Line of intersection between planes: (303867887720968, 410071000835863, 211451111726208, -3, 6, -3)
